In [2]:
import pandas as pd
import datetime
import math

#df = pd.read_csv('bc_trip.csv')
#df=df.drop(columns=['EVENT_NO_STOP'])
df=pd.read_csv('bc_trip.csv', usecols=['EVENT_NO_TRIP', 'OPD_DATE','VEHICLE_ID','METERS','ACT_TIME','GPS_LONGITUDE','GPS_LATITUDE'])
print(df)
df2=pd.DataFrame(data=df,columns = ['OPD_DATE'])

def getDecodedtDate(date):
    dt = datetime.datetime.strptime(date, "%d%b%Y:%H:%M:%S")
    return dt;
df2 = df2.applymap(getDecodedtDate)
df2= df2.rename(columns={'OPD_DATE': 'Date'})


df3=pd.DataFrame(data=df,columns = ['ACT_TIME'])
def getTimeStamp(time):
    ts=datetime.timedelta(seconds=time)
    return ts;
df3=df3.applymap(getTimeStamp)
df3= df3.rename(columns={'ACT_TIME': 'Time'})

df4 = pd.concat([df2, df3], axis=1)

df4 = df4.assign(DateTimeStamp = df4['Date'] + df4['Time'])
df['DateTimeStamp'] = pd.Series(df4['DateTimeStamp'])
 
# save the difference column to the dataframe

df['dMETERS'] = df.groupby('EVENT_NO_TRIP')['METERS'].diff()

df['dDateTimeStamp'] = df.groupby('EVENT_NO_TRIP')['DateTimeStamp'].diff()
calculateSpeed=  lambda x: round(x['dMETERS'] / x['dDateTimeStamp'].total_seconds(), 2) if not math.isnan(x['dMETERS']) and not math.isnan(x['dDateTimeStamp'].total_seconds()) else None
df['speed']=df.apply(calculateSpeed,axis=1)
df=df.drop(columns=['dMETERS','dDateTimeStamp'])
pd.set_option('display.float_format', '{:.2f}'.format)
df['speed'].describe()
#df.groupby('VEHICLE_ID')['speed'].describe()

#print(df.loc[df['speed'].idxmax()])

#print(df.groupby('VEHICLE_ID')['speed'].mean().idxmax())



     EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0        259172515  15FEB2023:00:00:00        4223      40     20469   
1        259172515  15FEB2023:00:00:00        4223      48     20474   
2        259172515  15FEB2023:00:00:00        4223      57     20479   
3        259172515  15FEB2023:00:00:00        4223      73     20484   
4        259172515  15FEB2023:00:00:00        4223     112     20489   
..             ...                 ...         ...     ...       ...   
156      259172515  15FEB2023:00:00:00        4223    5834     21389   
157      259172515  15FEB2023:00:00:00        4223    5838     21424   
158      259172515  15FEB2023:00:00:00        4223    5858     21429   
159      259172515  15FEB2023:00:00:00        4223    5889     21434   
160      259172515  15FEB2023:00:00:00        4223    5918     21439   

     GPS_LONGITUDE  GPS_LATITUDE  
0      -122.648137     45.493082  
1      -122.648240     45.493070  
2      -122.648352     45.4931

count   160.00
mean      7.23
std       4.42
min       0.00
25%       3.80
50%       6.40
75%      10.85
max      17.40
Name: speed, dtype: float64